In [13]:
import requests
games_URL = "https://api.nhle.com/stats/rest/en/game"

r = requests.get(url = games_URL)

games = r.json()

type(games)

import datetime

recentgames = []
for game in games['data']:
    if game['gameDate'] > '2022-01-01':
        recentgames.append(game)

In [66]:
print(games['data'][0])

{'id': 1917020001, 'easternStartTime': '1917-12-19T20:00:00', 'gameDate': '1917-12-19', 'gameNumber': 1, 'gameScheduleStateId': 1, 'gameStateId': 7, 'gameType': 2, 'homeScore': 4, 'homeTeamId': 36, 'period': 3, 'season': 19171918, 'visitingScore': 7, 'visitingTeamId': 8}


In [8]:
import requests
team_URL = "https://api.nhle.com/stats/rest/en/team"

r = requests.get(url = team_URL)

teams = r.json()

team_lib = {}
for team in teams['data']:
    team_lib[team['id']] = team['triCode']

print(team_lib)

{11: 'ATL', 34: 'HFD', 32: 'QUE', 33: 'WIN', 35: 'CLR', 36: 'SEN', 37: 'HAM', 38: 'PIR', 39: 'QUA', 40: 'DCG', 41: 'MWN', 42: 'QBD', 43: 'MMR', 44: 'NYA', 45: 'SLE', 46: 'OAK', 47: 'AFM', 48: 'KCS', 50: 'DFL', 51: 'BRK', 57: 'TAN', 58: 'TSP', 99: 'NHL', 17: 'DET', 6: 'BOS', 5: 'PIT', 14: 'TBL', 4: 'PHI', 12: 'CAR', 20: 'CGY', 8: 'MTL', 15: 'WSH', 23: 'VAN', 21: 'COL', 18: 'NSH', 24: 'ANA', 54: 'VGK', 25: 'DAL', 27: 'PHX', 16: 'CHI', 3: 'NYR', 13: 'FLA', 22: 'EDM', 30: 'MIN', 19: 'STL', 2: 'NYI', 26: 'LAK', 70: 'TBD', 7: 'BUF', 9: 'OTT', 10: 'TOR', 1: 'NJD', 52: 'WPG', 55: 'SEA', 28: 'SJS', 53: 'ARI', 59: 'UTA', 29: 'CBJ', 31: 'MNS', 49: 'CLE', 56: 'CGS'}


In [9]:
header = ['Home Team Previous Season Win Pctg', 'Home Team Current Season Win Pctg']

for i in range(1,19):
    header.append(f'Home Skater {i} point_avg')
    header.append(f'Home Skater {i} goal_avg')
    header.append(f'Home Skater {i} assist_avg')
    header.append(f'Home Skater {i} plusMinus_avg')
    header.append(f'Home Skater {i} toi_avg')
    header.append(f'Home Skater {i} shots_avg')
    
for i in range(1,3):
    header.append(f'Home Goalie {i} goals_against_avg')
    header.append(f'Home Goalie {i} save_pctg_avg')
    header.append(f'Home Goalie {i} goal_avg_avg')

header.append('Visiting Team Previous Season Win Pctg')
header.append('Visiting Team Current Season Win Pctg')

for i in range(1,19):
    header.append(f'Visiting Skater {i} point_avg')
    header.append(f'Visiting Skater {i} goal_avg')
    header.append(f'Visiting Skater {i} assist_avg')
    header.append(f'Visiting Skater {i} plusMinus_avg')
    header.append(f'Visiting Skater {i} toi_avg')
    header.append(f'Visiting Skater {i} shots_avg')
    
for i in range(1,3):
    header.append(f'Visiting Goalie {i} goals_against_avg')
    header.append(f'Visiting Goalie {i} save_pctg_avg')
    header.append(f'Visiting Goalie {i} goal_avg_avg')

header.append('Winner: 0 Home, 1Visiting')

In [10]:
import requests
from datetime import timedelta
import csv

def getpreviousskaterstats(season):
    prev_season_skater_stats_URL = f"https://api.nhle.com/stats/rest/en/skater/summary?cayenneExp=seasonId={season}"
    r = requests.get(url = prev_season_skater_stats_URL)
    prev_season_skater_stats = r.json()
    return prev_season_skater_stats['data']

def getpreviousgoaliestats(season):
    prev_season_goalie_stats_URL = f"https://api.nhle.com/stats/rest/en/goalie/summary?cayenneExp=seasonId={season}"
    r = requests.get(url = prev_season_goalie_stats_URL)
    prev_season_goalie_stats = r.json()
    return prev_season_goalie_stats['data']
    
def getpreviousteamstats(season):
    prev_season_team_stats_URL = f"https://api.nhle.com/stats/rest/en/team/summary?cayenneExp=seasonId={season}%20and%20gameTypeId=2"
    r = requests.get(url = prev_season_team_stats_URL)
    prev_season_team_stats = r.json()
    return prev_season_team_stats['data']

def getcurrentteamstats(teamid, date):
    prev_season_all_team_standing_URL = f"https://api-web.nhle.com/v1/standings/{date}"
    r = requests.get(url = prev_season_all_team_standing_URL)
    data = r.json()
    for team in data['standings']:
        if team['teamAbbrev']['default'] == team_lib[teamid]:
            return team
    return -1

def getteamrosters(gameid):
    team_roster_URL = f"https://api-web.nhle.com/v1/gamecenter/{gameid}/boxscore"
    r = requests.get(url = team_roster_URL)
    data = r.json()
    if data['homeTeam']['score'] > data['awayTeam']['score']:
        winner = 0
    else:
        winner = 1
    return data['playerByGameStats']['homeTeam'], data['playerByGameStats']['awayTeam'], winner

player_game_log_dict = {}

def clear_game_log_dict():
    player_game_log_dict = {}
    return 

def getcurrseasonskaterstats(playerid, season, game_date):
    data = None
    if f"{playerid}{season}" in player_game_log_dict:
        data = player_game_log_dict[f"{playerid}{season}"]
    else:
        player_gamelog_URL = f"https://api-web.nhle.com/v1/player/{playerid}/game-log/{season}/2"
        r = requests.get(url = player_gamelog_URL)
        data = r.json()
        player_game_log_dict[f"{playerid}{season}"] = data
    return summarizeskaterstats(data, game_date) 


def summarizeskaterstats(data, game_date):
    games = 0.0
    goals = 0.0
    assists = 0.0
    shots = 0.0
    points = 0.0
    plusMinus = 0.0
    toi = timedelta(0)
    #for game in data['gameLog']:
    i = len(data['gameLog']) - 1
    while data['gameLog'][i]['gameDate'] < game_date:
        game = data['gameLog'][i]
        games = games + 1
        goals = goals + game['goals']
        assists = assists + game['assists']
        points = points + game['points']
        shots = shots + game['shots']
        plusMinus = plusMinus + game['plusMinus']

        m1, s1 = map(int, game['toi'].split(":"))
        toi = toi + timedelta(minutes=m1, seconds=s1)
        
        i = i-1
        
    if games == 0:
        return 0,0,0,0,0,0
    point_avg = round(points/games,4)
    goal_avg = round(goals/games,4)
    assist_avg = round(assists/games,4)
    plusMinus_avg = round(plusMinus/games,4)
    shots_avg = round(shots/games,4)
    toi_avg = toi.total_seconds()/games
    return point_avg, goal_avg, assist_avg, plusMinus_avg, toi_avg, shots_avg

def getcurrseasongoaliestats(playerid, season, game_date):
    data = None
    if f"{playerid}{season}" in player_game_log_dict:
        data = player_game_log_dict[f"{playerid}{season}"]
    else:
        player_gamelog_URL = f"https://api-web.nhle.com/v1/player/{playerid}/game-log/{season}/2"
        r = requests.get(url = player_gamelog_URL)
        data = r.json()
        player_game_log_dict[f"{playerid}{season}"] = data
    return summarizegoaliestats(data, game_date) 

def summarizegoaliestats(data, game_date):
    games = 0.0
    goals_against = 0.0
    save_pctg = 0.0
    toi = timedelta(0)
    i = len(data['gameLog']) - 1
    while data['gameLog'][i-1]['gameDate'] < game_date:
        game = data['gameLog'][i]
        games = games + 1
        goals_against = goals_against + game['goalsAgainst']
        save_pctg = save_pctg + game['savePctg']

        m1, s1 = map(int, game['toi'].split(":"))
        toi = toi + timedelta(minutes=m1, seconds=s1)
        
        i = i-1
        
    if games == 0:
        return 0,0,0
    goals_against_avg = round(goals_against/games,4)
    save_pctg_avg = round(save_pctg/games,4)
    toi_avg = toi.total_seconds()/games
    return goals_against_avg, save_pctg_avg, toi_avg

In [17]:
import csv
import time

prev_season_player_stats = None
prev_season_goalie_stats = None
prev_season_team_stats = None
currentseason = '20092010'
start_time = time.time()
with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    exceptions = 0
    k=0
    prev_season_all_team_stats = None
    for game in recentgames:
        try:
            row = []
            if game['season'] != currentseason:
                prevseason = currentseason
                currentseason = game['season']
                prev_season_all_team_stats = {}
                prev_season_all_team_stats_raw = getpreviousteamstats(prevseason)
                home_team_prev_stats = None
                #clear_game_log_dict()
                
                for team in prev_season_all_team_stats_raw:
                    prev_season_all_team_stats[f"{team['teamId']}"] = team
            
            home_team = prev_season_all_team_stats[str(game['homeTeamId'])]
            win_pctg = round(home_team['wins']/home_team['gamesPlayed'],4)
            home_team_prev_stats = win_pctg
    
            home_team_roster, visiting_team_roster, winner = getteamrosters(game['id'])
            
            row.append(home_team_prev_stats)
    
    
            home_team_curr_stats = getcurrentteamstats(game['homeTeamId'], game['gameDate'])
            row.append(home_team_curr_stats['winPctg'])
            
            home_team_skaters_stats = []
            for forward in home_team_roster['forwards']:
                forward_stats = getcurrseasonskaterstats(forward['playerId'], currentseason,game['gameDate'])
                for stat in forward_stats:
                    row.append(stat)
                
            
            for defensemen in home_team_roster['defense']:
                defensemen_stats = getcurrseasonskaterstats(defensemen['playerId'], currentseason,game['gameDate'])
                for stat in defensemen_stats:
                    row.append(stat)
                
            
            home_team_goalie_stats = []
            for goalies in home_team_roster['goalies']:
                goalies_stats = getcurrseasongoaliestats(goalies['playerId'], currentseason, game['gameDate'])
                for stat in goalies_stats:
                    row.append(stat)
                
            
            
            
            #Visiting Team
            visiting_team = prev_season_all_team_stats[str(game['visitingTeamId'])]
            win_pctg = round(visiting_team['wins']/visiting_team['gamesPlayed'],4)
            visiting_team_prev_stats = win_pctg        
            row.append(visiting_team_prev_stats)
            
            visiting_team_curr_stats = getcurrentteamstats(game['visitingTeamId'], game['gameDate'])
            row.append(visiting_team_curr_stats['winPctg'])
            
            visiting_team_skaters_stats = []
            for forward in visiting_team_roster['forwards']:
                forward_stats = getcurrseasonskaterstats(forward['playerId'], currentseason,game['gameDate'])
                for stat in forward_stats:
                    row.append(stat)
                
            
            for defensemen in visiting_team_roster['defense']:
                defensemen_stats = getcurrseasonskaterstats(defensemen['playerId'], currentseason,game['gameDate'])
                for stat in defensemen_stats:
                    row.append(stat)
                
            
            for goalies in visiting_team_roster['goalies']:
                goalies_stats = getcurrseasongoaliestats(goalies['playerId'], currentseason,game['gameDate'])
                for stat in goalies_stats:
                    row.append(stat)
            row.append(winner)
            #write
            writer.writerow(row)
            k = k+1
        except:
            exceptions = exceptions + 1
            k=k+1
        print(f"\rCurrent game date: {game['gameDate']}Total games process: {k}, Number of exceptions:{exceptions}", end='', flush=True)

end_time = time.time()
elapsed_time = end_time - start_time
print("\n")
print(f"Elapsed time: {elapsed_time} seconds")

Current game date: 2025-02-17Total games process: 5402, Number of exceptions:2484

Elapsed time: 8025.411949634552 seconds


In [4]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("data20102012.csv")
# Check for duplicates
duplicates = df[df.duplicated()]

# Display the duplicate rows
print("Duplicate rows:")
print(duplicates)

num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

Duplicate rows:
Empty DataFrame
Columns: [Home Team Previous Season Win Pctg, Home Team Current Season Win Pctg, Home Skater 1 point_avg, Home Skater 1 goal_avg, Home Skater 1 assist_avg, Home Skater 1 plusMinus_avg, Home Skater 1 toi_avg, Home Skater 1 shots_avg, Home Skater 2 point_avg, Home Skater 2 goal_avg, Home Skater 2 assist_avg, Home Skater 2 plusMinus_avg, Home Skater 2 toi_avg, Home Skater 2 shots_avg, Home Skater 3 point_avg, Home Skater 3 goal_avg, Home Skater 3 assist_avg, Home Skater 3 plusMinus_avg, Home Skater 3 toi_avg, Home Skater 3 shots_avg, Home Skater 4 point_avg, Home Skater 4 goal_avg, Home Skater 4 assist_avg, Home Skater 4 plusMinus_avg, Home Skater 4 toi_avg, Home Skater 4 shots_avg, Home Skater 5 point_avg, Home Skater 5 goal_avg, Home Skater 5 assist_avg, Home Skater 5 plusMinus_avg, Home Skater 5 toi_avg, Home Skater 5 shots_avg, Home Skater 6 point_avg, Home Skater 6 goal_avg, Home Skater 6 assist_avg, Home Skater 6 plusMinus_avg, Home Skater 6 toi_avg, 